In [1]:
%run base.ipynb

In [2]:
J = Kinematics.J # maps body velocity to Euler rates
J_inv = Kinematics.J_INV # maps body velocity to Euler rates
Jdot = Kinematics.J_dot # maps body acceleration to Euler acceleration

In [3]:
x_sym = ca.vertcat(n ,x_nb, dx_nb)

nx = x_sym.size1()
print(f"State size: {nx}")

ny = 7
print(f"Measurement size: {ny}")

State size: 18
Measurement size: 7


In [4]:
# Unpack state
v     = x_nb              # body twist [u, v, w, p, q, r] in your convention
a     = dx_nb             # body twist acceleration

# First and second time derivatives of n
n_dot     = J @ v
n_dotdot  = J @ a + Jdot @ v

# Discrete dynamics, constant acceleration over the step
n_next = n + n_dot*dt + 0.5*n_dotdot*(dt**2)
v_next = v + a*dt
a_next = a                        # zero jerk model

x_pred = ca.vertcat(n_next, v_next, a_next)

# State Prediction and State Jacobian A = ∂f/∂x for EKF linearization
AF  = ca.jacobian(x_pred, x_sym)

In [5]:
# Define the measurement function.
# the measurements are: 
# [z, roll, pitch, yaw, dvl_vx, dvl_vy, dvl_vz] 
h_expr = ca.vertcat(
    x_sym[2],        # pressure depth z
    x_sym[3],        # imu roll
    x_sym[4],        # imu pitch
    x_sym[5],        # imu yaw
    x_sym[6],       # DVL velocity x
    x_sym[7],       # DVL velocity y
    x_sym[8]        # DVL velocity z
)

# Compute the measurement Jacobian: H = ∂h/∂x
H_sym = ca.jacobian(h_expr, x_sym)

# a function for the measurement model and its Jacobian
h_func = ca.Function('h_func', [x_sym], [h_expr, H_sym])

In [6]:
# --- EKF iteration as a Casadi function ---
y_sym = ca.SX.sym('y', ny)       # measurement vector
P_sym = ca.SX.sym('P', nx, nx)   # error covariance

# Process and measurement noise covariances
Q_sym = ca.SX.sym('Q', nx, nx)   # process noise covariance
R_sym = ca.SX.sym('R', ny, ny)   # measurement noise covariance

# Prediction step:
#   x_pred = f(x,u,dt)
#   P_pred = AF * P * AF^T + Q
P_pred = AF @ P_sym @ AF.T + Q_sym

# Measurement update:
#   Compute the predicted measurement and its Jacobian
h_pred, H = h_func(x_pred)

# Innovation (measurement residual)
Y_corrected_k = ca.diag([1,  
                         1, -1, -1, 
                         1, 1, 1])@y_sym #corrected z and yaw directions


y_err = Y_corrected_k - h_pred
# Innovation covariance
S = H @ P_pred @ H.T + R_sym
# Kalman gain
K = P_pred @ H.T @ ca.inv(S)
# Updated state estimate
x_upd = x_pred + K @ y_err
# Updated covariance
P_upd = (ca.SX.eye(nx) - K @ H) @ P_pred

# Create the EKF update function.
ekf_update = ca.Function('ekf_update',
                           [x_sym, P_sym, dt, y_sym, Q_sym, R_sym, v_c],
                           [x_upd, P_upd, y_err],
                           ['x', 'P','dt', 'y', 'Q', 'R', 'v_c'],
                           ['x_next', 'P_next', 'y_err'])
# ekf_update.save('EKF_update.casadi')

# c , cpp or matlab code generation for forward dynamics
ekf_update.generate("EKFnext.c")
os.system(f"gcc -fPIC -shared EKFnext.c -o libEKF_next.so")

0

In [7]:
ekf_update(x=ca.DM.ones(nx), P=ca.DM.eye(nx), dt=0.1, y=ca.DM.zeros(ny), Q=ca.DM.eye(nx)*0.1, R=ca.DM.eye(ny)*0.1, v_c=ca.DM.zeros(6))

{'P_next': DM(
 [[1.10645, -0.0044546, -5.13898e-05, 0.00848712, -0.000394859, -0.0111933, 0.00199724, -0.000322989, 0.00807279, -0.0035412, 0.00372522, 0.00056371, -0.000539769, -3.62174e-05, -0.00330343, -0.000424356, 0.000326168, 0.000185968], 
  [-0.0044546, 1.10586, 7.75034e-05, -0.00816889, -0.000622929, 0.0102809, 0.0041782, 0.00715252, -0.000779071, 0.00346315, -0.00294857, -0.000854001, -0.0019017, -0.00271553, 0.000417565, 0.000408444, -0.000248475, -0.000196558], 
  [-5.13898e-05, 7.75034e-05, 0.0917118, -0.000365569, -0.0011003, -6.67388e-05, -0.000573699, 0.0003518, 0.000156573, 0.000213511, 0.000549117, -0.000371208, 0.000224983, -0.000163388, -3.55947e-05, 1.82784e-05, 5.88759e-05, -2.75758e-05], 
  [0.00848712, -0.00816889, -0.000365569, 0.0911861, 0.0023402, -0.000118314, -2.53041e-05, 1.55168e-05, 6.90593e-06, 0.0088067, 0.0126897, 0.00586106, 9.9233e-06, -7.20652e-06, -1.56997e-06, 0.000440696, 0.000523531, 0.000475209], 
  [-0.000394859, -0.000622929, -0.0011003, 0.